In [7]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import time

DATASET_ROOT = "projects\data"
IMG_SIZE = 64
LATENT_DIM = 128
BATCH_SIZE = 64
EPOCHS = 3000

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

In [9]:
print(os.listdir(DATASET_ROOT))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'projects\\data'

In [ ]:
def load_images_multiclass(root):
    images = []

    for folder in os.listdir(root):
        folder_path = os.path.join(root, folder)
        if not os.path.isdir(folder_path):
            continue

        for file in os.listdir(folder_path):
            if not file.lower().endswith(".jpg"):
                continue

            img_path = os.path.join(folder_path, file)

            try:
                img = Image.open(img_path).convert("RGB")
            except:
                continue

            img = img.resize((IMG_SIZE, IMG_SIZE))
            img = np.array(img)
            img = (img - 127.5) / 127.5
            images.append(img)

    return np.array(images)

data = load_images_multiclass(DATASET_ROOT)
print("Total images:", data.shape[0])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(2000).batch(BATCH_SIZE).prefetch(1)

In [ ]:
def build_generator():
  m = tf.keras.Sequential()
  m.add(tf.keras.layers.Dense(8 * 8 * 128, input_dim=LATENT_DIM))
  m.add(tf.keras.layers.Reshape((8, 8, 128)))
  m.add(tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same'))
  m.add(tf.keras.layers.BatchNormalization())
  m.add(tf.keras.layers.ReLU())
  m.add(tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same'))
  m.add(tf.keras.layers.BatchNormalization())
  m.add(tf.keras.layers.ReLU())
  m.add(tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh'))
  return m

def build_descriminator():
  m = tf.keras.Sequential()
  m.add(tf.keras.layers.Conv2D(32, 4, strides=2, padding="same", input_shape=(64, 64, 3)))
  m.add(tf.keras.layers.LeakyReLU(0.2))
  m.add(tf.keras.layers.Conv2D(64, 4, strides=2, padding="same"))
  m.add(tf.keras.layers.LeakyReLU(0.2))
  m.add(tf.keras.layers.Flatten())
  m.add(tf.keras.layers.Dense(1))
  return m

generator = build_generator()
discriminator = build_descriminator()

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
opt_g = tf.keras.optimizers.Adam(0.0002, 0.5)
opt_d = tf.keras.optimizers.Adam(0.0001, 0.5)

@tf.function
def train_step(real_imgs):
  bs =  tf.shape(real_imgs)[0]
  noise = tf.random.normal([bs, LATENT_DIM])

  with tf.GradientTape() as d_tape:
    fake_imgs = generator(noise, training=True)
    real_logits = discriminator(real_imgs, training=True)
    fake_logits = discriminator(fake_imgs, training=True)
    d_loss_real = loss_fn(tf.ones_like(real_logits) * 0.9, real_logits)
    d_loss_fake = loss_fn(tf.zeros_like(fake_logits), fake_logits)
    d_loss = d_loss_real + d_loss_fake

  grads_d = d_tape.gradient(d_loss, discriminator.trainable_variables)
  opt_d.apply_gradients(zip(grads_d, discriminator.trainable_variables))

  for _ in range(2):
    noise = tf.random.normal([bs, LATENT_DIM])
    with tf.GradientTape() as tape:
      fake_imgs = generator(noise, training=True)
      fake_logits = discriminator(fake_imgs, training=True)
      g_loss =  loss_fn(tf.ones_like(fake_logits), fake_logits)
    grads = tape.gradient(g_loss, generator.trainable_variables)
    opt_g.apply_gradients(zip(grads, generator.trainable_variables))

  return d_loss, g_loss

In [ ]:
if not os.path.exists("output"):
  os.makedirs("output")

In [ ]:
def save_grid(generator, epoch, n=4):
  noise = tf.random.normal([n*n, LATENT_DIM])
  imgs = generator(noise, training=False)
  imgs = (imgs * 127.5 + 127.5).numpy().astype(np.uint8)

  grid = Image.new("RGB", (IMG_SIZE*n, IMG_SIZE*n))
  idx = 0
  for i in range(n):
    for j in range(n):
      grid.paste(Image.fromarray(imgs[idx]), (j*IMG_SIZE, i*IMG_SIZE))
      idx += 1

  grid.save("output/grid_" + str(epoch) + ".png")

In [ ]:
d_losses = []
g_losses = []

start_time = time.time()

for epoch in range(EPOCHS):
  for batch in dataset:
    d_loss, g_loss = train_step(batch)

  if epoch % 100 == 0:
    print("epoch", epoch, "D", float(d_loss), "G", float(g_loss))
    noise = tf.random.normal([1, LATENT_DIM])
    img = generator(noise, training=False)[0].numpy()
    img = (img * 127.5 + 127.5).astype(np.uint8)
    Image.fromarray(img).save("output/sample_" + str(epoch) + ".png")

  if epoch % 1000 == 0:
    generator.save("output/generator_" + str(epoch) + ".keras")
    discriminator.save("output/discriminator_" + str(epoch) + ".keras")

  if epoch % 500 == 0:
    save_grid(generator, epoch)

  d_losses.append(float(d_loss))
  g_losses.append(float(g_loss))

  print("Training time (minutes):", (time.time() - start_time) / 60)

In [ ]:
generator.save("output/generator_final.keras")

In [ ]:
img_check = (data[0] * 127.5 + 127.5).astype(np.uint8)
Image.fromarray(img_check)